In [1]:
import os
import gzip
import sys
import time
import pickle
import numpy as np
import theano.tensor as T
from theano import function, shared, pp
import theano

In [13]:
class LogisticRegression():
    def __init__(self, input, input_dimensions, output_dimensions):
        self.weights = shared(
            value=np.zeros(
                (input_dimensions, output_dimensions),
                dtype=theano.config.floatX
            ),
            name='weights',
            borrow=True
        )
        self.bias = shared(
            value=np.zeros(
                (output_dimensions,),
                dtype=theano.config.floatX
            ),
            name='bias',
            borrow=True
        )
        self.p_label_given_features = T.nnet.softmax(T.dot(input, self.weights) + self.bias)
        self.label_prediction = T.argmax(self.p_label_given_features, axis=1)
        self.params = [self.weights, self.bias]
    
    def negative_log_likelihood(self, labels):
        # T.log(self.p_label_given_features) is a matrix where each row is an example
        # and each column is a class, and each element is the log probability of that
        # example being in that class.
        # T.log(self.p_label_given_features)[T.arange(labels.shape[0]),labels)]
        # is a vector where each element is the log probability of the correct label given
        # the model.
        return -T.mean(T.log(self.p_label_given_features)[T.arange(labels.shape[0]), labels])
    
    def errors(self, labels):
        if labels.ndim != self.label_prediction.ndim:
            raise TypeError("labels should have the same shape as self.label_prediction",
                            ('labels', labels.type, 'label_prediction', self.label_prediction.type))
        if labels.dtype.startswith('int'):
            return T.mean(T.neq(self.label_prediction, labels))
        else:
            raise NotImplementedError()

# Copy and pasted. Not tryna deal with this.          
def load_data(dataset):
    ''' Loads the dataset

    :type dataset: string
    :param dataset: the path to the dataset (here MNIST)
    '''

    #############
    # LOAD DATA #
    #############

    # Download the MNIST dataset if it is not present
    data_dir, data_file = os.path.split(dataset)
    if data_dir == "" and not os.path.isfile(dataset):
        # Check if dataset is in the data directory.
        new_path = os.path.join(
            os.path.split(os.path.realpath('__file__'))[0],
            "data",
            dataset
        )
        if os.path.isfile(new_path) or data_file == 'mnist.pkl.gz':
            dataset = new_path

    if (not os.path.isfile(dataset)) and data_file == 'mnist.pkl.gz':
        import urllib
        origin = (
            'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        )
        print('Downloading data from %s' % origin)
        urllib.request.urlretrieve(origin, dataset)

    print('... loading data')

    # Load the dataset
    f = gzip.open(dataset, 'rb')
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
    f.close()
    #train_set, valid_set, test_set format: tuple(input, target)
    #input is an numpy.ndarray of 2 dimensions (a matrix)
    #witch row's correspond to an example. target is a
    #numpy.ndarray of 1 dimensions (vector)) that have the same length as
    #the number of rows in the input. It should give the target
    #target to the example with the same index in the input.

    def shared_dataset(data_xy, borrow=True):
        """ Function that loads the dataset into shared variables

        The reason we store our dataset in shared variables is to allow
        Theano to copy it into the GPU memory (when code is run on GPU).
        Since copying data into the GPU is slow, copying a minibatch everytime
        is needed (the default behaviour if the data is not in a shared
        variable) would lead to a large decrease in performance.
        """
        data_x, data_y = data_xy
        shared_x = theano.shared(np.asarray(data_x,
                                            dtype=theano.config.floatX),
                                 borrow=borrow)
        shared_y = theano.shared(np.asarray(data_y,
                                            dtype=theano.config.floatX),
                                 borrow=borrow)
        # When storing data on the GPU it has to be stored as floats
        # therefore we will store the labels as ``floatX`` as well
        # (``shared_y`` does exactly that). But during our computations
        # we need them as ints (we use labels as index, and if they are
        # floats it doesn't make sense) therefore instead of returning
        # ``shared_y`` we will have to cast it to int. This little hack
        # lets ous get around this issue
        return shared_x, T.cast(shared_y, 'int32')

    test_set_x, test_set_y = shared_dataset(test_set)
    valid_set_x, valid_set_y = shared_dataset(valid_set)
    train_set_x, train_set_y = shared_dataset(train_set)

    rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y),
            (test_set_x, test_set_y)]
    return rval

def nth_batch(data_set, batch_size, index):
    return data_set[index * batch_size : (1 + index) * batch_size]

def sgd_optimization_mnist(learning_rate=0.13, epochs=1000, dataset='mnist.pkl.gz', batch_size=600): 
    datasets = load_data(dataset)
    
    train_set_features, train_set_labels = datasets[0]
    validation_set_features, validation_set_labels = datasets[1]
    test_set_features, test_set_labels = datasets[2]
    
    train_batches_count = train_set_features.get_value(borrow=True).shape[0] // batch_size
    validation_batches_count = validation_set_features.get_value(borrow=True).shape[0] // batch_size
    train_batches_count = train_set_features.get_value(borrow=True).shape[0] // batch_size
    
    print("Building the model...")
    
    index=T.lscalar()
    
    data_features = T.matrix('data_features')
    data_labels = T.ivector('data_labels')

    classifier = LogisticRegression(input=data_features, input_dimensions=28*28, output_dimensions=10)

    cost = classifier.negative_log_likelihood(data_labels)
    
    test_model = function(
        inputs=[index],
        outputs=classifier.errors(data_labels),
        givens={
            data_features: nth_batch(test_set_features, batch_size, index),
            data_labels: nth_batch(test_set_labels, batch_size, index)
        }
    )
    
    validate_model = function(
        inputs=[index],
        outputs=classifier.errors(data_labels),
        givens={
            data_features: nth_batch(validation_set_features, batch_size, index),
            data_labels: nth_batch(validation_set_labels, batch_size, index)
        }
    )
    
    weights_gradient = T.grad(cost=cost, wrt=classifier.weights)
    bias_gradient = T.grad(cost=cost, wrt=classifier.bias)

    updates = [(classifier.weights, classifier.weights - learning_rate * weights_gradient),
               (classifier.bias, classifier.bias - learning_rate * bias_gradient)]
    
    train_model = function(
        inputs=[index],
        outputs=cost,
        updates=updates,
        givens={
            data_features: nth_batch(train_set_features, batch_size, index),
            data_labels: nth_batch(train_set_labels, batch_size, index)
        }
    )
    
    # print("cost:")
    # theano.printing.debugprint(cost)
    # print("train_model:")
    # theano.printing.debugprint(train_model)

    
sgd_optimization_mnist()

... loading data
Building the model...
cost:
Elemwise{neg,no_inplace} [@A] ''   
 |Elemwise{true_div,no_inplace} [@B] ''   
   |Sum{acc_dtype=float64} [@C] ''   
   | |AdvancedSubtensor [@D] ''   
   |   |Elemwise{log,no_inplace} [@E] ''   
   |   | |Softmax [@F] ''   
   |   |   |Elemwise{add,no_inplace} [@G] ''   
   |   |     |dot [@H] ''   
   |   |     | |data_features [@I]
   |   |     | |weights [@J]
   |   |     |DimShuffle{x,0} [@K] ''   
   |   |       |bias [@L]
   |   |ARange [@M] ''   
   |   | |TensorConstant{0} [@N]
   |   | |Subtensor{int64} [@O] ''   
   |   | | |Shape [@P] ''   
   |   | | | |data_labels [@Q]
   |   | | |Constant{0} [@R]
   |   | |TensorConstant{1} [@S]
   |   |data_labels [@Q]
   |Subtensor{int64} [@T] ''   
     |Elemwise{Cast{float64}} [@U] ''   
     | |Shape [@V] ''   
     |   |AdvancedSubtensor [@D] ''   
     |Constant{0} [@W]
train_model:
Elemwise{TrueDiv}[(0, 0)] [@A] ''   15
 |Sum{acc_dtype=float64} [@B] ''   14
 | |CrossentropySoftmaxArgma